In [1]:
import dask.dataframe as dd
from dask.distributed import Client
import numpy as np
import pandas as pd
import sys, os

# Khởi tạo Dask client để tối ưu hiệu suất
client = Client()

file_path = "single_file.csv"
output_files = ["file1.csv", "file2.csv", "file3.csv"]
# Tỉ lệ chia (3:2:1)

# Cấu hình
chunksize = 100000  # Đọc từng phần (tuỳ vào RAM)
temp_dir = "FL_Data1/"  # Thư mục lưu file tạm

output_files = [temp_dir + output_file for output_file in output_files]
print(output_files)

header_saved=False
for chunk in pd.read_csv(file_path, chunksize=chunksize):
    df_1 = chunk.sample(frac=0.4, random_state=42)
    df_rest = chunk.drop(df_1.index)
    df_2 = df_rest.sample(frac=0.5, random_state=42)
    df_3 = df_rest.drop(df_2.index)

    df_1.to_csv(output_files[0], mode="a", header=not header_saved, index=False)
    df_2.to_csv(output_files[1], mode="a", header=not header_saved, index=False)
    df_3.to_csv(output_files[2], mode="a", header=not header_saved, index=False)
    header_saved = True


['FL_Data1/file1.csv', 'FL_Data1/file2.csv', 'FL_Data1/file3.csv']


In [3]:
import dask.dataframe as dd
from dask.distributed import Client
import numpy as np
import pandas as pd
import sys, os

# Khởi tạo Dask client để tối ưu hiệu suất
client = Client()

file_path = "single_file.csv"
output_files = ["file1.csv", "file2.csv", "file3.csv"]
temp_dir = "FL_Data2/"  # Thư mục lưu file tạm
output_files = [temp_dir + output_file for output_file in output_files]
print(output_files)

# Định nghĩa tỉ lệ mong muốn cho từng file (label 0: label 1)
target_ratios = {
    output_files[0]: (463594, 1191181),  # File 1: label 0: label 1 = 1:3
    output_files[1]: (338276, 1233132),  # File 2: label 0: label 1 = 1:5
    output_files[2]: (296325, 1575683)   # File 3: label 0: label 1 = 3:2
}

# Cấu hình
chunksize = 100000  # Đọc từng phần (tuỳ vào RAM)
header_saved = False

for chunk in pd.read_csv(file_path, chunksize=chunksize):
    # Tách dữ liệu theo nhãn (giả sử cột nhãn là 'label')
    label_0 = chunk[chunk['label'] == 0]
    label_1 = chunk[chunk['label'] == 1]

    # Tính tổng số mẫu trong chunk
    total_samples = len(chunk)
    total_label_0 = len(label_0)
    total_label_1 = len(label_1)

    if total_label_0 == 0 or total_label_1 == 0:
        print(f"Chunk {chunk.index[0]} không có đủ nhãn để chia!")
        continue

    # Tính số lượng mẫu cần lấy cho từng file dựa trên tỉ lệ
    file_samples = {file: {'label_0': 0, 'label_1': 0} for file in output_files}

    # Phân bổ mẫu theo tỉ lệ
    remaining_label_0 = total_label_0
    remaining_label_1 = total_label_1

    for file, (ratio_0, ratio_1) in target_ratios.items():
        # Tính tỷ lệ tổng (ratio_0 + ratio_1)
        total_ratio = ratio_0 + ratio_1
        # Ước lượng số mẫu cho file này dựa trên tỉ lệ tổng
        file_size = int(total_samples / len(output_files))  # Phân bổ đều ban đầu
        label_0_count = int(file_size * (ratio_0 / total_ratio))
        label_1_count = int(file_size * (ratio_1 / total_ratio))

        # Điều chỉnh để không vượt quá số lượng mẫu còn lại
        label_0_count = min(label_0_count, remaining_label_0)
        label_1_count = min(label_1_count, remaining_label_1)

        file_samples[file]['label_0'] = label_0_count
        file_samples[file]['label_1'] = label_1_count
        remaining_label_0 -= label_0_count
        remaining_label_1 -= label_1_count

    # Phân bổ phần còn lại (nếu có) để đảm bảo không bỏ sót mẫu
    for file in output_files:
        if remaining_label_0 > 0:
            file_samples[file]['label_0'] += 1
            remaining_label_0 -= 1
        if remaining_label_1 > 0:
            file_samples[file]['label_1'] += 1
            remaining_label_1 -= 1

    # Lấy mẫu ngẫu nhiên từ từng nhóm nhãn
    df_1 = pd.concat([label_0.sample(n=file_samples[output_files[0]]['label_0'], random_state=42),
                      label_1.sample(n=file_samples[output_files[0]]['label_1'], random_state=42)])
    df_2 = pd.concat([label_0.sample(n=file_samples[output_files[1]]['label_0'], random_state=42),
                      label_1.sample(n=file_samples[output_files[1]]['label_1'], random_state=42)])
    df_3 = pd.concat([label_0.sample(n=file_samples[output_files[2]]['label_0'], random_state=42),
                      label_1.sample(n=file_samples[output_files[2]]['label_1'], random_state=42)])

    # Lưu dữ liệu vào file
    df_1.to_csv(output_files[0], mode="a", header=not header_saved, index=False)
    df_2.to_csv(output_files[1], mode="a", header=not header_saved, index=False)
    df_3.to_csv(output_files[2], mode="a", header=not header_saved, index=False)
    header_saved = True

# Đóng client Dask
client.close()

/home/th/miniconda3/envs/tf/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42387 instead
  warnings.warn(


['FL_Data2/file1.csv', 'FL_Data2/file2.csv', 'FL_Data2/file3.csv']
